In [ ]:
from larcv import larcv
larcv.load_pyutil
larcv.load_cvutil

import cv2
import ROOT
from ROOT import fcllite
from ROOT import geo2d
from ROOT import larocv
pdraw = geo2d.PyDraw()

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# make a fake TRACK image...
a=np.zeros((485,485))

trk_ctor = ROOT.std.vector(ROOT.cv.Point_('int'))()

trk_ctor.clear()
    
pt_x=[]
pt_y=[]

trk_ctor.push_back(ROOT.cv.Point_("int")(25,225))
trk_ctor.push_back(ROOT.cv.Point_("int")(50,225))
trk_ctor.push_back(ROOT.cv.Point_("int")(50,229))
trk_ctor.push_back(ROOT.cv.Point_("int")(29,229))
trk_ctor.push_back(ROOT.cv.Point_("int")(35,260))
trk_ctor.push_back(ROOT.cv.Point_("int")(33,260))
trk_ctor.push_back(trk_ctor[0])
for pt in trk_ctor:
    pt_x.append(pt.x)
    pt_y.append(pt.y)
    plt.plot(pt_x,pt_y,'o-',color='black',lw=1)

ax=plt.gca()
#ax.set_xlim(0,450)
#ax.set_ylim(0,450)
plt.grid()
plt.show()

# make a path representing this contour
path_list1 = []
for pt in trk_ctor:
    path_list1.append((pt.x,pt.y)) 
    

In [ ]:
path1 = matplotlib.path.Path(path_list1)
path2 = matplotlib.path.Path(path_list2)

img1 = path1.contains_points(np.row_stack(np.where(a==0)).T).reshape(485,485).astype(np.uint8)
img4 = path2.contains_points(np.row_stack(np.where(b==0)).T).reshape(485,485).astype(np.uint8)

###################TRACK IMAGE
img1=np.where(img1>0,img1+80,0)
img2 = img1.copy()
img3 = img1.copy()

M = cv2.getRotationMatrix2D((485/2,485/2),20,1)
#rotate 1
img1 = cv2.warpAffine(img1,M,(485,485))

#get nonzero for 1, and move it...
nzero=np.row_stack(np.where(img1>0)).T
img1[nzero[:,0],nzero[:,1]]=0
img1[nzero[:,0]+20,nzero[:,1]+75]=80

M = cv2.getRotationMatrix2D((485/2,485/2),45,1)
#rotate 2
img2 = cv2.warpAffine(img2,M,(485,485))

#flip 2
img2 = img2[:,::-1]

#get nonzero for 2, and move it...
nzero=np.row_stack(np.where(img2>0)).T
img2[nzero[:,0],nzero[:,1]]=0
img2[nzero[:,0]+20,nzero[:,1]-179]=80

#rotate 3
M = cv2.getRotationMatrix2D((485/2,485/2),45,1)
img3 = cv2.warpAffine(img3,M,(485,485))

#get nonzero for 3, and move it...
nzero=np.row_stack(np.where(img3>0)).T
img3[nzero[:,0],nzero[:,1]]=0
img3[nzero[:,0]+100,nzero[:,1]+148]=80

trk_img = img1+img2+img3

plt.imshow(trk_img,cmap='Greys',interpolation='none')
ax=plt.gca()
fig=plt.gcf()
fig.set_size_inches(20,12)

plt.rc('grid', linestyle="-", color='black')

mjx = mjy = 10.;
mix = miy = 1.

major_ticks_x = np.arange(0,485+mjx/2,mjx)                                                                                                                                                                                    
major_ticks_y = np.arange(0,485+mjy/2,mjy)                                                                                                                                                                                    
minor_ticks_x = np.arange(0+mix/2,485+mix*1.5,1)                                                                                                                                                                            
minor_ticks_y = np.arange(0+miy/2,485+miy*1.5,1)                                                                                                                                                                            

ax.set_xticks(major_ticks_x)                                                                                                                                                                                            
ax.set_yticks(major_ticks_y)                                                                                                                                                                                            
ax.grid(which='minor')                                                                                                                                                                                                  
ax.set_xticklabels(major_ticks_x.astype(np.uint16),rotation=45,fontsize=10,minor=False)                                                                                                                                                   
ax.set_yticklabels(major_ticks_y.astype(np.uint16),rotation=45,fontsize=10,minor=False) 

plt.grid()
plt.show()

In [ ]:
slt = larocv.SingleLinearTrack()

In [ ]:
plt.imshow(trk_img,cmap='Greys',interpolation='none')
ax=plt.gca()
fig=plt.gcf()
fig.set_size_inches(20,12)

plt.rc('grid', linestyle="-", color='black')

mjx = mjy = 10.;
mix = miy = 1.

major_ticks_x = np.arange(0,485+mjx/2,mjx)                                                                                                                                                                                    
major_ticks_y = np.arange(0,485+mjy/2,mjy)                                                                                                                                                                                    
minor_ticks_x = np.arange(0+mix/2,485+mix*1.5,1)                                                                                                                                                                            
minor_ticks_y = np.arange(0+miy/2,485+miy*1.5,1)                                                                                                                                                                            

ax.set_xticks(major_ticks_x)                                                                                                                                                                                            
ax.set_yticks(major_ticks_y)                                                                                                                                                                                            
ax.grid(which='minor')                                                                                                                                                                                                  
ax.set_xticklabels(major_ticks_x.astype(np.uint16),rotation=45,fontsize=10,minor=False)                                                                                                                                                   
ax.set_yticklabels(major_ticks_y.astype(np.uint16),rotation=45,fontsize=10,minor=False) 


mat = pdraw.mat(trk_img.astype(np.float32))
ctors=larocv.FindContours(mat)
colors=['red','green','blue']
for ix,ctor in enumerate(ctors):
    pt1=ROOT.cv.Point_("float")()
    pt2=ROOT.cv.Point_("float")()
    slt.EdgesFromMeanValue(ctor,pt1,pt2)
    ctor = [[pt.x,pt.y] for pt in ctor]
    ctor.append(ctor[0])
    ctor=np.array(ctor)
    ax.plot(ctor[:,0],ctor[:,1],lw=2,color=colors[ix])
    ax.plot(pt1.x,pt1.y,'*',color='yellow',markersize=20,alpha=0.7)
    ax.plot(pt2.x,pt2.y,'o',color='yellow',markersize=20,alpha=0.7)
    
plt.grid()
plt.show()